## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [0]:
!pip3 install tensorflow==2.0 --quiet

In [0]:
import tensorflow as tf

In [0]:
#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default


### Collect Data

In [3]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [0]:
import pandas as pd

In [0]:
data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/prices.csv')

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Check all columns in the dataset

In [7]:
data.columns

Index(['date', 'symbol', 'open', 'close', 'low', 'high', 'volume'], dtype='object')

### Drop columns `date` and  `symbol`

In [0]:
data.drop(['date', 'symbol'], axis=1, inplace=True)

In [9]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [10]:
data = data.iloc[0:1000]
data.shape

(1000, 5)

In [11]:
data['volume'] = data['volume']/(1000000)
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2.1636
1,125.239998,119.980003,119.940002,125.540001,2.3864
2,116.379997,114.949997,114.930000,119.739998,2.4895
3,115.480003,116.620003,113.500000,117.440002,2.0063
4,117.010002,114.970001,114.089996,117.330002,1.4086


### Divide the data into train and test sets

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X = data.drop(['volume'], axis=1)
Y = data['volume']

In [0]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.30, random_state=99)

#### Convert Training and Test Data to numpy float32 arrays


In [0]:
X_train = X_train.astype('float32')
X_train_array = X_train.values

In [0]:
Y_train = Y_train.astype('float32')
Y_train_array = Y_train.values

In [0]:
X_test = X_test.astype('float32')
X_test_array = X_test.values

In [0]:
Y_test = Y_test.astype('float32')
Y_test_array = Y_test.values

### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [0]:
from sklearn.preprocessing import Normalizer
Norm = Normalizer()
X_train_norm =  Norm.fit_transform(X_train_array) 
X_test_norm =  Norm.fit_transform(X_test_array) 

## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [0]:
#We are initializing weights and Bias with Zero
w = tf.zeros(shape=(4,1))
b = tf.zeros(shape=(1))

2.Define a function to calculate prediction

In [0]:
def prediction(x, w, b):
    
    xw_matmul = tf.matmul(x, w)
    y = tf.add(xw_matmul, b)
    
    return y

3.Loss (Cost) Function [Mean square error]

In [0]:
def loss(y_actual, y_predicted):
    
    diff = y_actual - y_predicted
    sqr = tf.square(diff)
    avg = tf.reduce_mean(sqr)
    
    return avg

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [0]:
def train(x, y_actual, w, b, learning_rate=0.01):
    
    #Record mathematical operations on 'tape' to calculate loss
    with tf.GradientTape() as t:
        
        t.watch([w,b])
        
        current_prediction = prediction(x, w, b)
        current_loss = loss(y_actual, current_prediction)
    
    #Calculate Gradients for Loss with respect to Weights and Bias
    dw, db = t.gradient(current_loss,[w, b])
    
    #Update Weights and Bias
    w = w - learning_rate*dw
    b = b - learning_rate*db
    
    return w, b

## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Train loss at every 5th iteration

In [33]:
#Train for 100 Steps
for i in range(100):
    
    w, b = train(X_train_norm, Y_train_array, w, b, learning_rate=0.01)
    print('Current Loss on iteration', i, loss(Y_train_array, prediction(X_train_norm, w, b)).numpy())

Current Loss on iteration 0 285.41675
Current Loss on iteration 1 283.37912
Current Loss on iteration 2 281.50146
Current Loss on iteration 3 279.77097
Current Loss on iteration 4 278.17612
Current Loss on iteration 5 276.7063
Current Loss on iteration 6 275.35193
Current Loss on iteration 7 274.10352
Current Loss on iteration 8 272.953
Current Loss on iteration 9 271.8927
Current Loss on iteration 10 270.91547
Current Loss on iteration 11 270.01483
Current Loss on iteration 12 269.1849
Current Loss on iteration 13 268.42
Current Loss on iteration 14 267.71524
Current Loss on iteration 15 267.06543
Current Loss on iteration 16 266.46677
Current Loss on iteration 17 265.91492
Current Loss on iteration 18 265.40652
Current Loss on iteration 19 264.9377
Current Loss on iteration 20 264.50577
Current Loss on iteration 21 264.10773
Current Loss on iteration 22 263.7409
Current Loss on iteration 23 263.40277
Current Loss on iteration 24 263.0913
Current Loss on iteration 25 262.80417
Current

### Get the shapes and values of W and b

In [43]:
print("W=", w)

W= tf.Tensor(
[[1.3038638]
 [1.3074802]
 [1.2901567]
 [1.319012 ]], shape=(4, 1), dtype=float32)


In [42]:
print("B=", b)

B= tf.Tensor([2.6104], shape=(1,), dtype=float32)


### Model Prediction on 1st Examples in Test Dataset

In [52]:
prediction(X_test_norm[0:1], w, b)

<tf.Tensor: id=1967522, shape=(1, 1), dtype=float32, numpy=array([[5.220546]], dtype=float32)>

## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load the given Iris data using pandas (Iris.csv)

In [0]:
Iris_data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Iris.csv')

### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [89]:
Iris_data = pd.get_dummies(Iris_data, columns=['Species'])
Iris_data.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species_Iris-setosa,Species_Iris-versicolor,Species_Iris-virginica
0,1,5.1,3.5,1.4,0.2,1,0,0
1,2,4.9,3.0,1.4,0.2,1,0,0
2,3,4.7,3.2,1.3,0.2,1,0,0
3,4,4.6,3.1,1.5,0.2,1,0,0
4,5,5.0,3.6,1.4,0.2,1,0,0


### Splitting the data into feature set and target set

In [0]:
X2 = Iris_data.drop(columns=['Id', 'Species_Iris-setosa', 'Species_Iris-versicolor', 'Species_Iris-virginica'], axis=1)
Y2 = Iris_data[['Species_Iris-setosa', 'Species_Iris-versicolor', 'Species_Iris-virginica']]

In [0]:
X2_train, X2_test, Y2_train, Y2_test = train_test_split(X2, Y2, test_size=0.30, random_state=55)
X2_train_array = X2_train.values
Y2_train_array = Y2_train.values
X2_test_array = X2_test.values
Y2_test_array = Y2_test.values

###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [0]:
#Initialize Sequential model
model = tf.keras.models.Sequential()

#Add Dense Layer which provides 3 Outputs after applying softmax
model.add(tf.keras.layers.Dense(3, activation='softmax'))

#Comile the model
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

### Model Training 

In [112]:
model.fit(X2_train_array, Y2_train_array, 
          validation_data=(X2_test_array,Y2_test_array), 
          epochs=100)

Train on 105 samples, validate on 45 samples
Epoch 1/100
105/105 [==============================] - 0s 4ms/sample - loss: 2.1000 - accuracy: 0.3429 - val_loss: 1.8194 - val_accuracy: 0.3111
Epoch 2/100
105/105 [==============================] - 0s 226us/sample - loss: 1.7244 - accuracy: 0.3429 - val_loss: 1.5580 - val_accuracy: 0.3111
Epoch 3/100
105/105 [==============================] - 0s 223us/sample - loss: 1.4626 - accuracy: 0.3429 - val_loss: 1.2905 - val_accuracy: 0.3333
Epoch 4/100
105/105 [==============================] - 0s 217us/sample - loss: 1.2046 - accuracy: 0.3429 - val_loss: 1.1004 - val_accuracy: 0.3111
Epoch 5/100
105/105 [==============================] - 0s 217us/sample - loss: 1.0147 - accuracy: 0.3429 - val_loss: 0.9702 - val_accuracy: 0.3111
Epoch 6/100
105/105 [==============================] - 0s 222us/sample - loss: 0.8791 - accuracy: 0.3429 - val_loss: 0.8235 - val_accuracy: 0.3556
Epoch 7/100
105/105 [==============================] - 0s 225us/sample - lo

### Model Prediction

In [113]:
model.predict(X2_test_array)

array([[7.9188299e-01, 1.8555273e-01, 2.2564273e-02],
       [9.0545553e-01, 8.6565897e-02, 7.9786647e-03],
       [7.9979086e-01, 1.7377380e-01, 2.6435317e-02],
       [2.1223459e-02, 3.6374867e-01, 6.1502790e-01],
       [4.4852216e-02, 2.8318626e-01, 6.7196155e-01],
       [7.9705161e-01, 1.7986064e-01, 2.3087768e-02],
       [1.3344169e-02, 2.5451505e-01, 7.3214072e-01],
       [3.6839556e-02, 3.6423656e-01, 5.9892386e-01],
       [9.6698415e-01, 3.1251065e-02, 1.7646713e-03],
       [8.8023365e-01, 1.0865486e-01, 1.1111538e-02],
       [9.5104188e-01, 4.5419820e-02, 3.5382868e-03],
       [8.1670925e-02, 4.4540611e-01, 4.7292295e-01],
       [1.9972960e-02, 2.3714955e-01, 7.4287748e-01],
       [7.9188299e-01, 1.8555273e-01, 2.2564273e-02],
       [1.6687838e-02, 2.6130110e-01, 7.2201109e-01],
       [8.3863236e-02, 3.9945364e-01, 5.1668304e-01],
       [2.9710600e-02, 4.3213543e-01, 5.3815395e-01],
       [6.8699187e-01, 2.4747172e-01, 6.5536469e-02],
       [7.1114480e-02, 4.788

### Save the Model

In [0]:
model.save('Iris.h5')

In [0]:
sgd_optimizer = tf.keras.optimizers.SGD(lr=0.03)
model.compile(optimizer=sgd_optimizer, loss='mse')

### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?